In [ ]:
# 1. to handle the data
import pandas as pd
import numpy as np
from scipy import stats
import csv
import folium
from folium.plugins import HeatMap

# to visualize the data
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# To preprocess the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
# import iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# machine learning
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
#for classification tasks
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
# pipeline
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error,mean_squared_error,r2_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime
from sklearn.metrics import mean_absolute_error

# ignore warnings   
import warnings

In [ ]:
pd.set_option('display.max_rows', None)  # None means unlimited rows
pd.set_option('display.max_columns', None)  # None means unlimited columns

warnings.filterwarnings("ignore")

In [ ]:
godavari_df = pd.read_csv('INDUS_GODAVARI/samples.csv', delimiter=';')
indus_df = pd.read_csv('INDUS_GODAVARI/samples_indus.csv', delimiter=';')


# Concatenate the rows
merged_df = pd.concat([godavari_df, indus_df], ignore_index=True)

# Write the merged dataframe to a new CSV file
merged_df.to_csv('INDUS_GODAVARI/merged_samples.csv', index=False)

In [ ]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv("INDUS_GODAVARI/samples.csv", delimiter=";")
# Drop the 6th, 7th, and 9th columns
df = df.drop(df.columns[[5, 6, 8]], axis=1)


# Extract unique parameter names from the 5th column
unique_parameters = df.iloc[:, 4].unique()

# Create a dictionary to hold data for each parameter
param_data = {}
for param in unique_parameters:
    param_data[param] = df[df.iloc[:, 4] == param].iloc[:, 5]

# Create a DataFrame from the dictionary
param_df = pd.DataFrame(param_data)

# Merge the original DataFrame with the parameter DataFrame
merged_df = pd.concat([df.iloc[:, :-2], param_df, df.iloc[:, -1]], axis=1)
merged_df = merged_df.drop(merged_df.columns[4], axis=1)
merged_df.columns = ['Station', 'Date', 'Time', 'Index', 'Alk-Tot', 'TP', 'O2-Dis', 'BOD', 'TOTCOLI', 'TDS', 'pH', 'B-Dis', 'TEMP', 'H-T', 'Na-Dis', 'TURB', 'Ca-Dis', 'COD', 'SO4-Dis', 'Cl-Dis', 'Mg-Dis', 'NOxN', 'TKN', 'FDS', 'Q-Inst', 'FECALCOLI', 'NO3N', 'NH4N', 'TSS', 'NH3N', 'NO2N', 'F-Dis', 'SAR', 'K-Tot', 'DIELDRIN', 'ENDOSULFANII', '24DDT', 'Cd-Tot', 'As-Tot', 'Ni-Tot', 'Hg-Tot', 'Cu-Tot', '24D', 'ENDOSULFANI', 'Pb-Tot', 'Fe-Tot', 'Zn-Tot', 'Cr-Tot', 'DDT', 'ALDRIN', '44DDT', 'BHC-gamma', 'FECALSTREP', 'Pb-Dis', 'TS', 'Fe-Dis', 'Cd-Dis', 'As-Dis', 'Hg-Dis', 'Mn-Dis', 'DRP', 'K-Dis', 'HCO3', 'CO3', 'H-Ca', 'SiO2-Dis', 'Alk-Phen', 'O2-Dis-Sat', 'TRANS', 'Cr-Dis', 'Cu-Dis', 'Ni-Dis', 'Zn-Dis', 'Al-Dis', 'TOC', 'Quality']
# Print the transformed DataFrame
print(merged_df)


merged_df.to_csv('INDUS_GODAVARI/merged_file.csv', index=False)

In [ ]:
# Load the data
data = pd.read_csv('INDUS_GODAVARI/merged_file.csv')

# Remove rows with 'Unknown' values
data_known = data[data['Quality'] != 'Unknown']

plt.figure(figsize=(6,6))
# Pie plot
plt.figure(figsize=(8, 8))
data_known['Quality'].value_counts().plot.pie(autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Quality')
plt.ylabel('')
plt.show()

# Preprocessing
# Encode the 'Quality' column into numerical labels
label_encoder = LabelEncoder()
data_known['Quality'] = label_encoder.fit_transform(data_known['Quality'])

# Split the known data into features and target variable
X_known = data_known.drop(columns=['Quality'])
y_known = data_known['Quality']

# Define categorical and numeric features
categorical_features = X_known.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_known.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Create pipelines for preprocessing
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('numeric', numeric_pipeline, numeric_features),
    ('categorical', categorical_pipeline, categorical_features)
])

# Combine preprocessing with SMOTE
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
])

# Apply preprocessing and SMOTE
X_resampled, y_resampled = pipeline.fit_resample(X_known, y_known)

# Split resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
# with open('output.txt', 'w') as file:
#     # Write the desired content to the file
#     file.write(y_test.to_string(index=False))

# Evaluate the model
accuracy_known = accuracy_score(y_test, y_pred)
print("Accuracy on known data:", accuracy_known)
print("Classification Report on known data:")
print(classification_report(y_test, y_pred))

unknown_data = data[data['Quality'] == 'Unknown']
X_unknown = unknown_data.drop(columns=['Quality'])

# Apply preprocessing on unknown data
X_unknown_processed = pipeline[:-1].transform(X_unknown)

# Predict unknown values
unknown_predictions = model.predict(X_unknown_processed)
unknown_data['Predicted_Quality'] = label_encoder.inverse_transform(unknown_predictions)

# Save X_unknown and their predicted values into a new CSV file
unknown_data.to_csv('INDUS_GODAVARI/predicted_unknown.csv', index=False)
print("Predicted Quality for Unknown values saved in predicted_unknown.csv file.")


In [ ]:
# read data 
df = pd.read_csv("combinedData_Kaggle/water_potability.csv")
# print first 5 rows
df.head()

In [ ]:
# Target distribution
plt.figure(figsize=(6,6))
# Pie plot
df['Potability'].value_counts().plot.pie(explode=[0.1,0.1],
                    autopct='%1.1f%%', shadow=True,
                    textprops={'fontsize':16}).set_title("Target distribution")

In [ ]:
# Assuming df is your DataFrame
variables = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
             'Organic_carbon', 'Trihalomethanes', 'Turbidity']

# Set up the figure and axes
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))

# Flatten the axes for easy iteration
axes = axes.flatten()

# Loop through each variable and create histograms
for i, var in enumerate(variables):
    ax = axes[i]
    sns.histplot(df[var], kde=True, ax=ax)  
    ax.axvline(df[var].mean(), color='red', linestyle='--', label='Mean')
    ax.axvline(df[var].median(), color='blue', linestyle='--', label='Median')
    
    # Annotate plot with mean and median
    ax.annotate(f'Mean: {df[var].mean():.2f}\nMedian: {df[var].median():.2f}',
                xy=(0.05, 0.95), xycoords='axes fraction', ha='left', va='top')
    
    ax.set_title(f'Histogram with KDE for {var}')
    ax.set_xlabel(var)
    ax.legend()

# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
# now check the correlation of all columns
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(),annot=True,cmap='coolwarm',linewidths=0.4)

In [ ]:
def compute_dew_point(temperature, relative_humidity):
    T = temperature
    RH = relative_humidity / 100.0
    
    # Magnus formula constants
    a = 17.27
    b = 237.7
    
    # Calculate saturation vapor pressure
    alpha = ((a * T) / (b + T)) + np.log(RH)
    saturation_vapor_pressure = b * (np.exp(alpha) / (a - np.exp(alpha)))
    
    # Calculate dew point temperature
    dew_point_temperature = (b * alpha) / (a - alpha)
    
    return dew_point_temperature

In [ ]:
# Assuming merged_df is your DataFrame containing the features
merged_df = pd.read_csv('INDUS_GODAVARI/merged_file.csv')  # Load your dataset here

# Drop non-feature columns
features = merged_df.drop(columns=['Station', 'Date', 'Time', 'Quality'])

# Impute missing values using mean imputation
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

# Standardize the features
features_standardized = (features_imputed - features_imputed.mean()) / features_imputed.std()

# Perform PCA to reduce dimensionality
pca = PCA(n_components=2)  # You can adjust the number of components
pca_result = pca.fit_transform(features_standardized)

# Perform clustering
kmeans = KMeans(n_clusters=3)  # You can adjust the number of clusters
clusters = kmeans.fit_predict(pca_result)

# # Add cluster labels to the DataFrame
# merged_df['Cluster'] = clusters

# Group by 'Station' and compute the mean of 'TEMP' and 'H_T', ignoring NaN values
station_means = merged_df.groupby('Station')[['TEMP', 'H-T']].mean().dropna()

# Convert the result to a dictionary with station as key and (temperature, H_T) tuple as value
station_means_dict = station_means.apply(tuple, axis=1).to_dict()

# Compute dew point temperature for each sample using PCA components
dew_point_temperatures = []
for i in range(len(merged_df)):
    station=merged_df.at[i,'Station']
    if station in station_means_dict:
        temperature = station_means_dict[station][0]  # Assuming 'TEMP' is temperature column
        relative_humidity = station_means_dict[station][1] # Assuming 'H-T' is relative humidity column
        dew_point_temp = compute_dew_point(temperature, relative_humidity)
        dew_point_temperatures.append(dew_point_temp)
    else:   
        dew_point_temperatures.append(np.nan)
        
# Assuming dew_point_temperatures is your list of values
output_file_path = "INDUS_GODAVARI/dew_point_temperatures.txt"

with open(output_file_path, 'w') as f:
    for temperature in dew_point_temperatures:
        f.write(str(temperature) + '\n')


# Add predicted dew point temperatures to the DataFrame
merged_df['Predicted Dew Point'] = dew_point_temperatures
merged_df.to_csv('INDUS_GODAVARI/Predicted_dew.csv',index=False)

# Visualize explained variance ratio to decide on the number of components
plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.show()

# Visualize clusters in PCA space
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=clusters, cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA with Cluster Labels')
plt.show()


In [ ]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv('Canada/Canada.csv', encoding='latin1')

# Pivot the data to have each unique VARIABLE as a separate column
pivoted_data = data.pivot_table(index=['PROV_TERR','SITE_NO','SITE_NAME_NOM', 'DATE'],
                                columns='VARIABLE',
                                values='VALUE_VALEUR').reset_index()

# Select only the desired columns
selected_columns = ['PROV_TERR','SITE_NO','SITE_NAME_NOM','DATE', 'TEMPERATURE (WATER)', 'PH', 'TURBIDITY']

# Filter the pivoted data directly
filtered_data = pivoted_data[selected_columns]

columns_to_fill = ['TEMPERATURE (WATER)', 'PH', 'TURBIDITY']
for column in columns_to_fill:
    filtered_data[column].fillna(filtered_data[column].mean(), inplace=True)
    
filtered_data['DATE'] = pd.to_datetime(filtered_data['DATE'], dayfirst=True)
filtered_data = filtered_data.sort_values(by='DATE')

# Save the filtered data to a new CSV file
filtered_data.to_csv('Canada/filtered_canada_data.csv', index=False)


In [ ]:
# Read the CSV file into a DataFrame
data = pd.read_csv('Canada/filtered_canada_data.csv')
data=data.drop(columns=['DATE','PROV_TERR','SITE_NO'])

grouped_data = data.groupby('SITE_NAME_NOM').mean().reset_index()

# Draw box plot for Turbidity
plt.figure(figsize=(8, 6))
plt.boxplot(grouped_data['TURBIDITY'])
plt.title('Box Plot of Turbidity')
plt.ylabel('Turbidity')
plt.show()

# Draw box plot for WaterTemperature
plt.figure(figsize=(8, 6))
plt.boxplot(grouped_data['TEMPERATURE (WATER)'])
plt.title('Box Plot of Water Temperature')
plt.ylabel('Water Temperature')
plt.show()

# Draw box plot for Rainfall
plt.figure(figsize=(8, 6))
plt.boxplot(grouped_data['PH'])
plt.title('Box Plot of pH')
plt.ylabel('pH')
plt.show()

In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('Canada/filtered_canada_data.csv')

# Drop 'SITE_NO' and 'PROV_TERR'
df.drop(columns=['SITE_NO', 'PROV_TERR'], inplace=True)

# Convert 'DATE' column to datetime
df['DATE'] = pd.to_datetime(df['DATE'])

# Find the most frequent SITE_NAME_NOM
most_frequent_site = 'SALMON RIVER AT HYDER, ALASKA'

# Filter the DataFrame for the most frequent site
most_frequent_df = df[df['SITE_NAME_NOM'] == most_frequent_site]

# Plot TEMPERATURE (WATER), PH, and TURBIDITY individually as a function of time
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
sns.lineplot(x='DATE', y='TEMPERATURE (WATER)', data=most_frequent_df)
plt.title('Temperature (Water)')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')

plt.subplot(1, 3, 2)
sns.lineplot(x='DATE', y='PH', data=most_frequent_df)
plt.title('pH')
plt.xlabel('Date')
plt.ylabel('pH')

plt.subplot(1, 3, 3)
sns.lineplot(x='DATE', y='TURBIDITY', data=most_frequent_df)
plt.title('Turbidity')
plt.xlabel('Date')
plt.ylabel('Turbidity')

plt.tight_layout()
plt.show()


In [ ]:
df=pd.read_csv('Canada/filtered_canada_data.csv')
unique_values = df['SITE_NAME_NOM'].unique()
# print(unique_values)

In [ ]:
site_coordinates={
    'MERSEY R. BELOW MILL FALLS SW OF MAITLAND BRIDGE':[44.408160,-65.233150],
    'ASSINIBOINE RIVER AT HWY 8 BRIDGE':[46.289870,-66.592980],
    'BOW RIVER AT HIGHWAY 1 ABOVE LAKE LOUISE':[51.334970,-116.049740],
    'BOW RIVER ABOUT 4.5 KM ABOVE CANMORE':[51.088425,-115.350551],
    'CARROT RIVER NEAR TURNBERRY,':[53.279560,-103.587550],
    'ATHABASCA RIVER AT HIGHWAY #16 BELOW SNARING RIVER':[54.729170,-113.279200],
    'SALMON RIVER AT HYDER, ALASKA':[45.367690,-63.230620],
    'KOOTENAY RIVER AT KOOTENAY CROSSING':[49.102690,-116.554240],
    "BEAVER RIVER AT BEAVER CROSSING,":[44.014960,-66.149250],
    'WINNIPEG RIVER AT POINTE DU BOIS,':[50.294330,-95.544890],
    'PEMBINA RIVER AT WINDYGATES, MANITOBA':[52.105820,-101.266100],
    'RED RIVER AT EMERSON, MANITOBA':[49.717560,-97.126360],
    'LIARD RIVER AT UPPER CROSSING':[60.051760,-128.908480],
    'SOURIS RIVER NEAR WESTHOPE':[46.373180,-62.278660],
    'ILLECILLEWAET RIVER AT GLACIER NATIONAL PARK ENTRANCE':[-43.391920,170.181770],
    'LIARD RIVER AT FORT LIARD':[61.742670,-121.219670],
    'MACKENZIE RIVER ABOVE ARCTIC RED RIVER':[47.801200,-103.213180],
    'HAY RIVER NEAR ALBERTA/NWT BORDER':[60.810460,-115.788150],
    'LIARD RIVER NEAR THE MOUTH':[59.514120,-126.364610],
    'MACKENZIE RIVER AT STRONG POINT':[44.574490,-64.160770],
    'GREAT BEAR RIVER AT OUTLET OF GREAT BEAR LAKE':[27.889740,-80.511350],
    'MACKENZIE RIVER AT NORMAN WELLS':[65.270710,-126.765710],
    'SASKATCHEWAN RIVER ABOVE CARROT RIVER':[53.279560,-103.587550],
    'RED DEER RIVER NEAR BINDLOSS, ALBERTA':[46.140170,-77.553650],
    'SOUTH SASKATCHEWAN RIVER AT HWY 41,':[49.768070,-94.392340],
    'NORTH SASKATCHEWAN RIVER AT HIGHWAY #17 BRIDGE':[46.793590,-84.391110],
    'COLD RIVER AT OUTLET OF COLD LAKE':[44.622340,-84.923290],
    'SAINT-LAURENT À LÉVIS':[46.789440,-71.194760],
    'DES OUTAOUAIS À CARILLON':[45.682260,-76.622480],
    "QU'APPELLE RIVER APPROX. 3.2KM. SOUTH":[50.769610,-103.790310],
    'RED DEER RIVER AT ERWOOD,':[46.140170,-77.553650],
    'NORTH SASKATCHEWAN RIVER AT WHIRLPOOL POINT':[53.834740,-107.032540],
    'ATHABASCA RIVER ABOVE ATHABSCA FALLS':[57.182310,-111.633200],
    'CHURCHILL RIVER BELOW WASAWAKASIK':[55.918350,-107.719200],
    'KLONDIKE RIVER UPSTREAM OF BONANZA CREEK':[36.064950,-115.305440],
    'SOUTH MCQUESTEN RIVER DOWNSTREAM OF FLAT CREEK':[34.681080,-80.638930],
    'YUKON RIVER UPSTREAM OF TAKHINI RIVER':[60.876000,-135.435450],
    'SAINT-FRANÇOIS RIVER AT PIERREVILLE':[49.994050,-97.780910],
    'NICOLET RIVER AT NICOLET':[45.986650,-88.652580],
    'YAMASKA RIVER, HIGHWAY 132 BRIDGE':[45.258300,-66.089250],
    'ANNAPOLIS R. @ BRIDGE 650M SOUTH HWY 1 (WSC GAUGE)':[45.568870,-65.753310],
    'CORNWALLIS RIVER AT BRIDGE 850 M NORTH OF HWY 1':[49.185620,-122.750140],
    "ST. MARY'S RIVER AT HWY 7 BRIDGE,STILLWATER":[44.639920,-73.113450],
    'SOUTH RIVER AT ST. ANDREWS':[45.470610,-61.942790],
    'CHETICAMP RIVER ABOVE ROBERT BROOK (WSC GAUGE)':[46.644510,-60.947590],
    'KELLEY RIVER AT NINE MILE FORD, GAME SANCTUARY':[44.320680,-79.887020],
    'SHELBURNE RIVER':[42.627610,-72.736790],
    'TUSKET RIVER AT WILSONS BRIDGE':[51.940540,-114.639360],
    'ROSEWAY RIVER (WSC GAUGE), LOWER OHIO':[44.169675,-65.396104],
    'LAHAVE RIVER @ WEST NORTHFIELD BRIDGE (WSC GAUGE)':[51.172908,-115.568212],
    "ST. LAWRENCE RIVER, WATER INTAKE OF BÉCANCOUR'S FILTRATION PLANT":[44.617200,-75.405900],
    'LITTLE SACKVILLE RIVER':[45.892690,-64.975100],
    'NORTHEAST MARGAREE RIVER AT BRIDGE (WSC GAUGE)':[51.940540,-114.639360],
    'SACKVILLE RIVER AT HWY 1 BRIDGE, BEDFORD':[44.843470,-63.799180],
    "ST. LAWRENCE RIVER, WATER INTAKE OF BÉCANCOUR'S FILTRATION PLANT":[46.311578,-72.546012],
    "RICHELIEU RIVER, WATER INTAKE OF SOREL'S FILTRATION PLANT":[46.039305,-73.114843],
    "Fleuve Saint-Laurent, Prise d'eau de l'usine de filtration de Lavaltrie":[45.874908,-73.281964],
    "Rivière Saint-Maurice, Prise d'eau de l'usine de filtration de Trois-Rivières":[46.378640,-72.614220],
    'Rivière à la Pêche, station CABIN RPA001 (Parc national de la Mauricie)':[46.650760,-72.712080]
}


In [ ]:
# Read the original CSV file
with open('Canada/filtered_canada_data.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    rows = list(csv_reader)

# Modify the rows to include latitude and longitude
for row in rows:
    site_name = row['SITE_NAME_NOM']
    if site_name in site_coordinates:
        latitude, longitude = site_coordinates[site_name][0],site_coordinates[site_name][1]
        row['LATITUDE'] = latitude
        row['LONGITUDE'] = longitude
    else:
        row['LATITUDE'] = None
        row['LONGITUDE'] = None

# Write the modified data to a new CSV file
fieldnames = csv_reader.fieldnames + ['LATITUDE', 'LONGITUDE']
with open('Canada/modified_filtered_canada_data.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)


In [ ]:
# Read the CSV file into a DataFrame
data = pd.read_csv('Canada/modified_filtered_canada_data.csv')
data=data.drop(columns=['PROV_TERR','SITE_NO','DATE'])

# Group by 'SITE_NO' and calculate the mean for each parameter
grouped_data = data.groupby('SITE_NAME_NOM').mean().reset_index()

latitude_column = 'LATITUDE'  # Replace with your latitude column name
longitude_column = 'LONGITUDE'  # Replace with your longitude column name

# Create separate heatmaps for each parameter
parameters = ['TURBIDITY','PH','TEMPERATURE (WATER)']
for param in parameters:
    # Create a map centered at the mean latitude and longitude of all sites
    canada_map = folium.Map(location=[grouped_data[latitude_column].mean(), grouped_data[longitude_column].mean()], zoom_start=4)
    
    # Create a HeatMap layer for the parameter
    heat_data = [[row[latitude_column], row[longitude_column], row[param]] for index, row in grouped_data.iterrows()]
    HeatMap(heat_data, radius=10).add_to(canada_map)

    # Save the map as an HTML file
    canada_map.save(f'Canada/canada_heatmap_{param}.html')


In [ ]:
def extract_station_info(file_path):
    """
    Extracts station information from the initial lines of the file.
    """
    station_info = {}
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#Station'):
                key, value = line.strip().split(',', 1)
                station_info[key] = value
    return station_info

def filter_and_combine(file_paths):
    """
    Filters the datasets and combines them based on Timestamp.
    """
    # Extract station information for each dataset
    station_info = [extract_station_info(file_path) for file_path in file_paths]

    # Initialize an empty DataFrame to store combined data
    combined_data = pd.DataFrame(columns=['Station Number', 'Station Location Latitude', 'Station Location Longitude', 'Timestamp'])

    # Iterate over each dataset
    for i, file_path in enumerate(file_paths):
        # Load the dataset skipping initial lines
        data = pd.read_csv(file_path, skiprows=9)  # Skip 9 lines to reach data
        
        # Extract necessary columns
        filtered_data = data[['#Timestamp', 'Value']]
        
        # Add station information
        filtered_data['Station Number'] = station_info[i]['#Station Number']
        filtered_data['Station Location Latitude'] = station_info[i]['#Station Location Latitude']
        filtered_data['Station Location Longitude'] = station_info[i]['#Station Location Longitude']
        
        # Convert timestamp to date
        filtered_data['Timestamp'] = pd.to_datetime(filtered_data['#Timestamp']).dt.date
        
        # Rename 'Value' column to include dataset name
        dataset_name = file_path.split('/')[1]
        dataset_name = dataset_name.split('_')[0]
        filtered_data.rename(columns={'Value': f'{dataset_name}'}, inplace=True)
        
        # Merge with combined data based on Timestamp
        combined_data = pd.merge(combined_data, filtered_data, on=['Station Number', 'Station Location Latitude', 'Station Location Longitude', 'Timestamp'], how='outer')

    return combined_data

# List of file paths for each dataset
file_paths = ['Australia/Turbidity_Australia.csv', 'Australia/WaterTemperature_Australia.csv', 'Australia/Rainfall_Australia.csv']

# Filter and combine datasets
final_dataset = filter_and_combine(file_paths)

# Reorder columns
final_dataset = final_dataset[['Station Number', 'Station Location Latitude', 'Station Location Longitude', 'Timestamp', 'Turbidity', 'WaterTemperature', 'Rainfall']]

columns_to_fill = ['WaterTemperature']
for column in columns_to_fill:
    final_dataset[column].fillna(final_dataset[column].mean(), inplace=True)

# Replace missing values in 'Value_Rainfall' with mode
mode_rainfall = final_dataset['Rainfall'].mode()[0]
final_dataset['Rainfall'].fillna(mode_rainfall, inplace=True)
# Replace missing values in 'Value_Rainfall' with mode

median_turbidity = final_dataset['Turbidity'].median()
final_dataset['Turbidity'].fillna(median_turbidity, inplace=True)

# Save the final dataset
final_dataset.to_csv('Australia/Final_Dataset_Australia.csv', index=False)


In [ ]:
# Read the CSV file into a DataFrame
data = pd.read_csv('Australia/Final_Dataset_Australia.csv')
data=data.drop(columns=['Timestamp'])

# Group by 'SITE_NO' and calculate the mean for each parameter
grouped_data = data.groupby('Station Number').mean().reset_index()

latitude_column = 'Station Location Latitude'  # Replace with your latitude column name
longitude_column = 'Station Location Longitude'  # Replace with your longitude column name

# Create separate heatmaps for each parameter
parameters = ['Turbidity','WaterTemperature','Rainfall']
for param in parameters:
    # Create a map centered at the mean latitude and longitude of all sites
    australia_map = folium.Map(location=[grouped_data[latitude_column].mean(), grouped_data[longitude_column].mean()], zoom_start=4)
    
    # Create a HeatMap layer for the parameter
    heat_data = [[row[latitude_column], row[longitude_column], row[param]] for index, row in grouped_data.iterrows()]
    HeatMap(heat_data, radius=10).add_to(australia_map)

    # Save the map as an HTML file
    australia_map.save(f'Australia/heatmap_{param}.html')


In [ ]:
# Load the dataset
df = pd.read_csv('Australia/Final_Dataset_Australia.csv')

# Convert Timestamp column to datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Plot Turbidity, Water Temperature, and Rainfall individually
plt.figure(figsize=(10, 6))

# Plot Turbidity
plt.subplot(3, 1, 1)
plt.plot(df['Timestamp'], df['Turbidity'], color='blue')
plt.title('Turbidity')
plt.xlabel('Timestamp')
plt.ylabel('Turbidity')

# Plot Water Temperature
plt.subplot(3, 1, 2)
plt.plot(df['Timestamp'], df['WaterTemperature'], color='green')
plt.title('Water Temperature')
plt.xlabel('Timestamp')
plt.ylabel('Temperature (°C)')

# Plot Rainfall
plt.subplot(3, 1, 3)
plt.plot(df['Timestamp'], df['Rainfall'], color='orange')
plt.title('Rainfall')
plt.xlabel('Timestamp')
plt.ylabel('Rainfall (mm)')

plt.tight_layout()
plt.show()


In [ ]:
# Read the dataset
df = pd.read_csv('Australia/Final_Dataset_Australia.csv')

# Drop columns station number, latitude, and longitude
df.drop(['Station Number', 'Station Location Latitude', 'Station Location Longitude','Timestamp'], axis=1, inplace=True)

# Convert Timestamp to datetime if needed
# df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Draw box plot for Turbidity
plt.figure(figsize=(8, 6))
plt.boxplot(df['Turbidity'])
plt.title('Box Plot of Turbidity')
plt.ylabel('Turbidity')
plt.show()

# Draw box plot for WaterTemperature
plt.figure(figsize=(8, 6))
plt.boxplot(df['WaterTemperature'])
plt.title('Box Plot of Water Temperature')
plt.ylabel('Water Temperature')
plt.show()

# Draw box plot for Rainfall
plt.figure(figsize=(8, 6))
plt.boxplot(df['Rainfall'])
plt.title('Box Plot of Rainfall')
plt.ylabel('Rainfall')
plt.show()


In [ ]:
# Step 1: Load and preprocess the dataset
def load_dataset(file_path):
    dataset = pd.read_csv(file_path)
    # Drop unnecessary columns
    dataset.drop(['Station Number', 'Station Location Latitude', 'Station Location Longitude'], axis=1, inplace=True)
    return dataset

# Step 2: Organize data for LSTM
def prepare_data(dataset, target):
    # Convert date to datetime
    dataset['Timestamp'] = pd.to_datetime(dataset['Timestamp'])
    # Sort by date
    dataset.sort_values(by='Timestamp', inplace=True)
    
    # Extract features and target
    features = dataset['Timestamp']
    target_values = dataset[target]
    
    # Normalize target values
    scaler = MinMaxScaler(feature_range=(0, 1))
    target_values_scaled = scaler.fit_transform(np.array(target_values).reshape(-1, 1))
    
    return features, target_values_scaled, scaler

# Inside the train_lstm function, add a verbose mode to print the training loss
def train_lstm(features, target_values_scaled):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(features.shape[1], 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(features, target_values_scaled, epochs=100, batch_size=32, verbose=1)
    # Print training loss
    print("Training loss:", history.history['loss'][-1])
    return model

# Load dataset
dataset = load_dataset('Australia/Final_Dataset_Australia.csv')

# Define targets
targets = ['Turbidity', 'WaterTemperature', 'Rainfall']
models = {}
mae_results = {}

# Split dataset into train and test sets
train_size = 0.8  # 80% train, 20% test
train_idx = int(len(dataset) * train_size)
train_data, test_data = dataset.iloc[:train_idx], dataset.iloc[train_idx:]

# Train LSTM models for each target
for target in targets:
    features_train, target_values_scaled_train, scaler = prepare_data(train_data, target)
    features_train = np.array(features_train.apply(lambda x: x.value)).reshape(-1, 1, 1)
    lstm_model = train_lstm(features_train, target_values_scaled_train)
    models[target] = (lstm_model, scaler)
    
    # Prepare test data
    features_test, target_values_scaled_test, _ = prepare_data(test_data, target)
    features_test = np.array(features_test.apply(lambda x: x.value)).reshape(-1, 1, 1)
    # Make predictions
    predictions_scaled = lstm_model.predict(features_test)
    # Inverse scaling to get actual values
    predictions = scaler.inverse_transform(predictions_scaled)
    # Compute MAE
    mae = mean_absolute_error(test_data[target], predictions)
    mae_results[target] = mae


In [ ]:
# Step 4: Define function to predict target values
def predict_values(model, scaler, date):
    # Preprocess date
    date_timestamp = datetime.timestamp(date)
    date_array = np.array([[date_timestamp]])
    # Normalize date
    date_scaled = scaler.transform(date_array)
    print("Scaled input:", date_scaled)
    # Predict target value
    prediction_scaled = model.predict(date_scaled.reshape(1, 1, 1))
    print("Scaled prediction:", prediction_scaled)
    # Inverse scaling to get actual value
    prediction = scaler.inverse_transform(prediction_scaled)
    print("Inverse scaled prediction:", prediction)
    return prediction[0][0]

# Define function to predict values for a given date
def predict_values_for_date(date):
    predictions = {}
    for target, (model, scaler) in models.items():
        prediction = predict_values(model, scaler, date)
        predictions[target] = prediction
    return predictions

# Test the function
test_date = datetime.strptime('2012-09-05', '%Y-%m-%d')
predicted_values = predict_values_for_date(test_date)
print("Predicted values:", predicted_values)

# Print MAE results
print("\nMean Absolute Error:",)
for target, mae in mae_results.items():
    print(f"{target}: {mae}")
